# Data Cleaning Listado de Listado de Beneficiarios 2022

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_22 -> Dataset con el listado de TODOS los beneficiarios de 2022.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [ ]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [ ]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text)  #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2022

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [ ]:
beneficiarios_22 = pd.read_csv(
    '../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2022.csv', encoding='utf-8',
    skiprows=0)

In [ ]:
beneficiarios_22 = beneficiarios_22[~(
            (beneficiarios_22['ENTIDAD'] == 'NACIONAL') & (beneficiarios_22['MUNICIPIO'] == 'NACIONAL') & (
                beneficiarios_22['LOCALIDAD'] == 'NACIONAL'))]

In [ ]:
beneficiarios_22.shape

In [ ]:
nan_rows = beneficiarios_22[beneficiarios_22.isna().any(axis=1)]

In [ ]:
nan_rows

In [ ]:
beneficiarios_22['ENTIDAD'].unique()

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [ ]:
beneficiarios_22.dropna(inplace=True)

In [ ]:
# Obtenemos las localidades únicas en el dataset.
Localidades_22 = beneficiarios_22[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_22 = Localidades_22.drop_duplicates()


In [ ]:
Localidades_22['ENTIDAD_c_benef'] = Localidades_22['ENTIDAD'].apply(clean_text)
Localidades_22['MUNICIPIO_c_benef'] = Localidades_22['MUNICIPIO'].apply(clean_text)
Localidades_22['LOCALIDAD_c_benef'] = Localidades_22['LOCALIDAD'].apply(clean_text)

In [ ]:
# Creamos las dos keys de beneficiarios 2019
Localidades_22['KEY_benef_mun'] = Localidades_22['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_22[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_22['KEY_benef_loc'] = Localidades_22['MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_22['LOCALIDAD_c_benef'].astype(str)

In [ ]:
Localidades_22.shape

In [ ]:
# Obtenemos las localidades únicas en el dataset.
Municipios_22 = beneficiarios_22[['ENTIDAD', 'MUNICIPIO']]
Municipios_22 = Municipios_22.drop_duplicates()

In [ ]:
# Estandarizamos la limpieza de los datos
Municipios_22['ENTIDAD_c_benef'] = Municipios_22['ENTIDAD'].apply(clean_text)
Municipios_22['MUNICIPIO_c_benef'] = Municipios_22['MUNICIPIO'].apply(clean_text)

In [ ]:
# Creamos las dos keys de beneficiarios 
Municipios_22['KEY_benef_mun'] = Municipios_22['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_22[
    'MUNICIPIO_c_benef'].astype(str)

In [ ]:
Municipios_22.shape

In [ ]:
Municipios_22.columns

### 1.2 INEGI 2022

Obtener listado único de municipios y listado único de localidades de inegi de 2022 con sus claves.

In [ ]:
path_dataset_inegi_2022 = '../../data/inegi/dataset_inegi_clean_2022.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2022 = pd.read_csv(path_dataset_inegi_2022)

In [ ]:
dataset_inegi_2022['CVE_ENT'] = dataset_inegi_2022['CVE_ENT'].astype(str).str.zfill(2)
dataset_inegi_2022['CVE_MUN'] = dataset_inegi_2022['CVE_MUN'].astype(str).str.zfill(3)
dataset_inegi_2022['CVE_LOC'] = dataset_inegi_2022['CVE_LOC'].astype(str).str.zfill(4)

### 1.2.1 INEGI 2022 Municipios únicos para cada año.

In [ ]:
dataset_inegi_2022['KEY_inegi_municipio'] = dataset_inegi_2022['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2022['Municipio_c_inegi'].astype(str)
dataset_inegi_2022['KEY_inegi_localidad'] = dataset_inegi_2022['Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2022['Localidad_c_inegi'].astype(str)

In [ ]:
INEGI_UNIQUEMUN_2022 = dataset_inegi_2022.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad", "POB_TOTAL"])

INEGI_UNIQUEMUN_2022 = INEGI_UNIQUEMUN_2022.drop_duplicates()

In [ ]:
INEGI_UNIQUEMUN_2022.shape

In [ ]:
INEGI_UNIQUEMUN_2022.columns

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2022, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_22 y el dataset de INEGI correspondiente.

In [ ]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()

    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]

    df_benef['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))

    return df_merged

In [ ]:
diccionario_MUN_22 = fuzzy_merge_benef2019_2022(Municipios_22, INEGI_UNIQUEMUN_2022, 'KEY_benef_mun',
                                                'KEY_inegi_municipio')
diccionario_MUN_22.shape

In [ ]:
diccionario_MUN_22.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
                                 'Municipio_c_inegi'], inplace=True)
diccionario_MUN_22.drop_duplicates(subset='KEY_benef_mun', keep='first', inplace=True)

In [ ]:
diccionario_MUN_22.shape

In [ ]:
diccionario_MUN_22.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_22.csv',
                          index=False)
diccionario_MUN_22_simple = diccionario_MUN_22.drop(
    columns=['best_match', 'match_score', 'CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'])
# # TODO Es necesario modificar el diccionario manual
# diccionario_MUN_22_simple.to_csv(
#     '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_22_simple.csv', index=False)

In [ ]:
diccionario_MUN_22_simple = pd.read_csv(
    '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_22_simple.csv')

In [ ]:
beneficiarios_22.shape

In [ ]:
duplicates_specific_column = diccionario_MUN_22_simple[
    diccionario_MUN_22_simple.duplicated(subset=['KEY_benef_mun', 'KEY_inegi_municipio'], keep=False)]
print(duplicates_specific_column)

In [ ]:
duplicates_specific_column = diccionario_MUN_22_simple[
    diccionario_MUN_22_simple.duplicated(subset='KEY_inegi_municipio', keep=False)]
print(duplicates_specific_column)

In [ ]:
#diccionario_MUN_22_simple = diccionario_MUN_22_simple.drop_duplicates(subset='KEY_inegi_municipio', keep='first')

In [ ]:
#diccionario_MUN_22_simple = diccionario_MUN_22_simple.drop_duplicates(subset='KEY_benef_mun', keep='first')

In [ ]:
diccionario_MUN_22_simple.shape

In [ ]:
nan_rows = beneficiarios_22[beneficiarios_22.isna().any(axis=1)]

In [ ]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_22[beneficiarios_22['ENTIDAD'] == 'NACIONAL']

### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [ ]:
beneficiarios_22.columns

In [ ]:
beneficiarios_22.shape

In [ ]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_22['ESTADO_Clean'] = beneficiarios_22['ENTIDAD'].apply(clean_text)
beneficiarios_22['MUNICIPIO_Clean'] = beneficiarios_22['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_22['Estado-mun-KEY'] = beneficiarios_22['ESTADO_Clean'].astype(str) + '-' + beneficiarios_22[
    'MUNICIPIO_Clean'].astype(str)

In [ ]:
beneficiarios_22[beneficiarios_22['Estado-mun-KEY'] == 'chiapas-belisario dominguez']

In [ ]:
diccionario_MUN_22_simple[diccionario_MUN_22_simple['KEY_benef_mun'] == 'chiapas-palenque']

In [ ]:
diccionario_MUN_22_simple.drop_duplicates(inplace=True)

In [ ]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_22, diccionario_MUN_22_simple, left_on="Estado-mun-KEY",
                                         right_on="KEY_benef_mun", how='left', suffixes=('_benef', '_inegi'))

In [ ]:
listado_beneficiarios_parte_I.columns

In [ ]:
listado_beneficiarios_parte_I.shape

In [ ]:
print(INEGI_UNIQUEMUN_2022.shape)
INEGI_UNIQUEMUN_2022.drop_duplicates(subset='KEY_inegi_municipio', keep='first', inplace=True)
print(INEGI_UNIQUEMUN_2022.shape)

In [ ]:
# TODO: REVISAR ESTA PARTE (ARTURO)
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2022,
                                          left_on="KEY_inegi_municipio",
                                          right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [ ]:
listado_beneficiarios_parte_II.columns

In [ ]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(
    columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'KEY_inegi_municipio', 'Entidad_c_inegi', 'Estado-mun-KEY',
             'Municipio_c_inegi', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD',
             'ESLABÓN'])

In [ ]:
listado_beneficiarios_parte_II.columns

In [ ]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2022.csv', index=False)

In [ ]:
listado_beneficiarios_parte_II.shape

In [ ]:
listado_beneficiarios_parte_II.isna().sum()

In [ ]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_II[listado_beneficiarios_parte_II.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [ ]:
rows_with_nan

# 3. Join por entidad (Divide y Vencerás)

In [ ]:
listado_beneficiarios_2022 = pd.read_csv('../../data/listados_completos/listado_beneficiarios_2022.csv')

In [ ]:
listado_beneficiarios_2022.columns

In [ ]:
listado_beneficiarios_2022['ENTIDAD_c_benef'] = listado_beneficiarios_2022['ENTIDAD'].apply(clean_text)
listado_beneficiarios_2022['MUNICIPIO_c_benef'] = listado_beneficiarios_2022['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_2022['LOCALIDAD_c_benef'] = listado_beneficiarios_2022['LOCALIDAD'].apply(clean_text)

In [ ]:
listado_beneficiarios_2022['KEY_benef_loc'] = listado_beneficiarios_2022['MUNICIPIO_c_benef'].astype(str) + '-' + listado_beneficiarios_2022['LOCALIDAD_c_benef'].astype(str)

In [ ]:
listado_beneficiarios_2022.shape

In [ ]:
listado_beneficiarios_2022 = listado_beneficiarios_2022.drop_duplicates(subset=['KEY_benef_loc'])

In [ ]:
listado_beneficiarios_2022.shape

In [ ]:
listado_beneficiarios_2022.ENTIDAD.unique()

### 3.2.1 Join de localidades - GUERRERO

Empezamos creando los datasets por entiedad del dataset de beneficiarios.


In [ ]:
entities = ['GUERRERO', 'OAXACA', 'PUEBLA', 'MORELOS', 'CHIAPAS', 'DURANGO','TLAXCALA', 'NAYARIT', 'ZACATECAS']

# Dictionary to hold DataFrames
dataframes_dict = {}

for entity in entities:
    variable_name = f"Localidades_22_{entity.upper().replace(' ', '_')}"
    dataframe = listado_beneficiarios_2022[listado_beneficiarios_2022["ENTIDAD"] == entity]
    
    # Check if the DataFrame for this entity already exists in the dictionary
    if variable_name in dataframes_dict:
        # If it exists, concatenate the new DataFrame with the existing one
        dataframes_dict[variable_name] = pd.concat([dataframes_dict[variable_name], dataframe])
    else:
        # If it does not exist, create a new entry in the dictionary
        dataframes_dict[variable_name] = dataframe

# Define each DataFrame as a separate variable
for name, df in dataframes_dict.items():
    globals()[name] = df
    print(f"{name} defined with shape: {df.shape}")

In [ ]:
listado_beneficiarios_2022.shape

### 3.2.2 Diccionarios de Inegi por entidad.

In [ ]:
folder_path = '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/2022'

# Listar todos los archivos en la carpeta
files = os.listdir(folder_path)

# Filtrar solo los archivos CSV
csv_files = [file for file in files if file.endswith('.csv')]

# Leer cada archivo CSV y almacenar en variables separadas
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    # Crear un nombre de variable dinámicamente eliminando la extensión .csv
    var_name = file_name.replace('.csv', '')
    # Leer el archivo CSV y asignar el DataFrame a la variable
    globals()[var_name] = pd.read_csv(file_path)

# Opcional: Imprimir el nombre de las variables creadas y la cantidad de filas y columnas de cada DataFrame
for file_name in csv_files:
    var_name = file_name.replace('.csv', '')
    df = globals()[var_name]
    print(f'{var_name}: {df.shape[0]} filas, {df.shape[1]} columnas')


In [ ]:
Localidades_22_PUEBLA_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_PUEBLA, INEGI_UNIQUELOC_2022_puebla, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_PUEBLA_parte_I.shape

Localidades_22_MORELOS_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_MORELOS, INEGI_UNIQUELOC_2022_morelos, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_MORELOS_parte_I.shape

Localidades_22_TLAXCALA_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_TLAXCALA, INEGI_UNIQUELOC_2022_tlaxcala, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_TLAXCALA_parte_I.shape

Localidades_22_DURANGO_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_DURANGO, INEGI_UNIQUELOC_2022_durango, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_DURANGO_parte_I.shape

Localidades_22_OAXACA_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_OAXACA, INEGI_UNIQUELOC_2022_oaxaca, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_OAXACA_parte_I.shape

Localidades_22_NAYARIT_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_NAYARIT, INEGI_UNIQUELOC_2022_nayarit, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_NAYARIT_parte_I.shape

Localidades_22_CHIAPAS_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_CHIAPAS, INEGI_UNIQUELOC_2022_chiapas, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_CHIAPAS_parte_I.shape

Localidades_22_ZACATECAS_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_ZACATECAS, INEGI_UNIQUELOC_2022_zacatecas, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_ZACATECAS_parte_I.shape

Localidades_22_GUERRERO_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_GUERRERO, INEGI_UNIQUELOC_2022_guerrero, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_GUERRERO_parte_I.shape

In [ ]:
# Concatenate datasets vertically
diccionario_LOC_22 = pd.concat([
    Localidades_22_GUERRERO_parte_I,
    Localidades_22_PUEBLA_parte_I,
    Localidades_22_MORELOS_parte_I,
    Localidades_22_TLAXCALA_parte_I,
    Localidades_22_DURANGO_parte_I,
    Localidades_22_OAXACA_parte_I,
    Localidades_22_NAYARIT_parte_I,
    Localidades_22_CHIAPAS_parte_I,
    Localidades_22_ZACATECAS_parte_I
], axis=0)

# Check the shape of the concatenated DataFrame
diccionario_LOC_22.shape

In [ ]:
diccionario_LOC_22.columns

In [ ]:
diccionario_LOC_22.drop(columns=['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun',
       'CVE_ENT_benef', 'Entidad_inegi_benef', 'CVE_MUN_benef',
       'Municipio_inegi_benef', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'LOCALIDAD_c_benef', 'best_match',
       'CVE_ENT_inegi', 'Entidad_inegi_inegi', 'CVE_MUN_inegi',
       'Municipio_inegi_inegi', 'CVE_LOC', 'Localidad_inegi', 'POB_TOTAL',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'Localidad_c_inegi',], inplace=True)

In [ ]:
diccionario_LOC_22.columns

In [ ]:
diccionario_LOC_22.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_22.csv', index=False)

In [ ]:
diccionario_LOC_22.isna().sum()

# 4. Diccionarios de localidades y merge respectivos.

Empezamos leyendo el diccionario_LOC_22_simple, el cual solo tiene key_benef_mun y key_inegi_municipio manipulada.

In [ ]:
import pandas as pd

# Intentar leer el archivo CSV con diferentes configuraciones de lectura
try:
    diccionario_LOC_22_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_22_simple.csv', encoding='cp1252', 
                          delimiter=';')
    #diccionario_LOC_22_simple.drop(columns=['ValidaciÃ³n', 'Propuesta'], inplace=True)
    print(diccionario_LOC_22_simple.head())
except pd.errors.ParserError as e:
    print("Error al leer el archivo CSV:", e)




In [ ]:
diccionario_LOC_22_simple.columns

In [ ]:
diccionario_LOC_22_simple.rename(columns={'ï»¿KEY_benef_loc': 'KEY_benef_loc'}, inplace=True)

In [ ]:
diccionario_LOC_22_simple.columns

In [ ]:
diccionario_LOC_22_simple

In [ ]:
listado_beneficiarios_parte_II['MUNICIPIO_Clean'] = listado_beneficiarios_parte_II['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_parte_II['LOCALIDAD_Clean'] = listado_beneficiarios_parte_II['LOCALIDAD'].apply(clean_text)

# Create KEY in listado beneficiarios
listado_beneficiarios_parte_II['Municipio-loc-KEY'] = listado_beneficiarios_parte_II['MUNICIPIO_Clean'].astype(str) + '-' + listado_beneficiarios_parte_II[
    'LOCALIDAD_Clean'].astype(str)

Primer merge entre listado_beneficiarios_parte_II, y diccionario_LOC_22_simple. En este merge cogemos el dataset con los municipios corregidos previamente y ejecutamos el merge de manera que nos traemos las key_inegi_localidad corregidas.

In [ ]:
listado_beneficiarios_parte_II.shape

In [ ]:
diccionario_LOC_22_simple.drop_duplicates(inplace=True)

In [ ]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I_localidades = pd.merge(listado_beneficiarios_parte_II, diccionario_LOC_22_simple, left_on="Municipio-loc-KEY",
                                         right_on="KEY_benef_loc", how='left', suffixes=('_benef', '_inegi'))

In [ ]:
# Marcar todos los duplicados en las columnas 'columna1' y 'columna2' como True
duplicados = listado_beneficiarios_parte_I_localidades.duplicated()

# Filtrar el DataFrame original para mostrar solo los duplicados
df_duplicados = listado_beneficiarios_parte_I_localidades[duplicados]

In [ ]:
listado_beneficiarios_parte_I_localidades.shape

In [ ]:
df_duplicados

In [ ]:
listado_beneficiarios_parte_I_localidades = listado_beneficiarios_parte_I_localidades.drop_duplicates()

In [ ]:
listado_beneficiarios_parte_I_localidades.shape

In [ ]:
dataset_inegi_2022.columns

In [ ]:
INEGI_UNIQUELOC_2022 = dataset_inegi_2022.drop(columns=['Entidad_c_inegi', 'Municipio_c_inegi',
       'Localidad_c_inegi', 'POB_TOTAL'])
INEGI_UNIQUELOC_2022 = INEGI_UNIQUELOC_2022.drop_duplicates(subset='KEY_inegi_localidad', keep='first')

In [ ]:
INEGI_UNIQUELOC_2022.shape

In [ ]:
listado_beneficiarios_parte_I_localidades.columns

In [ ]:
listado_beneficiarios_parte_I_localidades.isna().sum()

In [ ]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_I_localidades[listado_beneficiarios_parte_I_localidades.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [ ]:
rows_with_nan

In [ ]:
# TODO: REVISAR ESTA PARTE (ARTURO)
listado_beneficiarios_parte_II_localidades = pd.merge(listado_beneficiarios_parte_I_localidades, INEGI_UNIQUELOC_2022,
                                          left_on="KEY_inegi_localidad",
                                          right_on="KEY_inegi_localidad", how='left', suffixes=('_benef', '_inegi'))

In [ ]:
listado_beneficiarios_parte_II_localidades.isna().sum()

In [ ]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_II_localidades[listado_beneficiarios_parte_II_localidades.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [ ]:
# Suponiendo que quieres verificar NaNs en la columna 'nombre_columna'
rows_with_nan = listado_beneficiarios_parte_II_localidades[listado_beneficiarios_parte_II_localidades['CVE_ENT_inegi'].isna()]

In [ ]:
rows_with_nan[rows_with_nan['CVE_ENT_inegi'].isna()]

In [ ]:
listado_beneficiarios_parte_II_localidades.drop(columns=['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'KEY_benef_mun',
       'CVE_ENT_benef', 'Entidad_inegi_benef', 'CVE_MUN_benef',
       'Municipio_inegi_benef', 'MUNICIPIO_Clean', 'LOCALIDAD_Clean',
       'Municipio-loc-KEY', 'KEY_benef_loc', 'match_score',
       'KEY_inegi_localidad', 'KEY_inegi_municipio'
       ], inplace=True)

In [ ]:
listado_beneficiarios_parte_II_localidades.rename(columns={'BENEFICIARIO': 'Nombre del beneficiario',
                                                           'ZONA': 'Zona del país',
                                                           'CVE_ENT_inegi': 'Clave de entidad',
                                                            'Entidad_inegi_inegi': 'Entidad federativa',
                                                            'CVE_MUN_inegi': 'Clave de municipio',
                                                            'Municipio_inegi_inegi': 'Municipio',                                                           
                                                            'CVE_LOC': 'Clave de localidad',
                                                            'Localidad_inegi': 'Localidad',
                                                            'PRODUCTO': 'Producto cultivado',
                                                            'FECHA': 'Fecha de entrega del beneficio',
                                                            'MONTO FEDERAL': 'Monto entregado',
                                                            'CICLO AGRÍCOLA': 'Ciclo agrícola'}, inplace=True)

In [ ]:
listado_beneficiarios_parte_II_localidades.shape

In [ ]:
listado_beneficiarios_parte_II_localidades.to_csv('../../data/listados_completos/listado_beneficiarios_2022_localidades.csv', index=False)